# Scikit learn cheat sheet

Cheat sheet to scikit learn. Includes

- Pre-processing data
- Models for classification
- Models for regression (incomplete)
- Pipelines

Sources:
 - Lecture slides, Uni Michgigan Applied Data Science with Python
 - Stack Overflow
 - Others, see comments

ver 0.02, in progress... 

# Data sets and imports

In [1]:
# Jupyter lab
# To get this working: https://github.com/matplotlib/jupyter-matplotlib
%matplotlib widget
import matplotlib.pyplot as plt

In [2]:
# Jupyter notebooks
#%matplotlib notebook
#import matplotlib.pyplot as plt

In [3]:
from sklearn import datasets
import seaborn as sns
import pandas as pd
import numpy as np
from IPython.display import display_html
from IPython.display import display

def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    html_str = html_str.replace('table','table style="display:inline; border:0px"')          
    display_html(html_str,raw=True)

iris = datasets.load_iris()

In [4]:
random_state = 3

# Classification

## Plotters for classification

In [5]:
def plot_decision_boundaries(X, y, model, title = None):
    '''
    Takes as inputs 
        X_test (two features)
        y_test
        fitted model
        
    To do: Needs to be made dynamice w.r.t to target classes
    '''
    no_targets = len(y.unique())
    
    mesh_step_size = 0.01
    x_min, x_max = X[:, 0].min() - .1, X[:, 0].max() + .1
    y_min, y_max = X[:, 1].min() - .1, X[:, 1].max() + .1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, mesh_step_size), np.arange(y_min, y_max, mesh_step_size))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Colors    
    import matplotlib.colors as mcolors
    colors = sns.color_palette("husl", no_targets)
    cmap = mcolors.LinearSegmentedColormap.from_list("Custom", colors, len(colors))
    

    # Figure
    fig = plt.figure(figsize = (6,4), dpi = 100)
    ax = fig.add_subplot(111)
    scats = []
    for i in range(no_targets):
        ax.scatter(X[y.values == i,0], X[y.values == i,1], alpha = 0.8, label = i, color = colors[i], s = 10)
    plt.imshow(Z, interpolation = 'nearest', cmap = cmap, alpha = 0.15,
               extent=(x_min, x_max, y_min, y_max), origin = 'lower')
    ax.legend()
    #ax.set_aspect('equal')
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)    
    if title:
        ax.set_title(title)
    

In [6]:
def plot_bin_decision_probs(X, y, model, title = None):
    '''
    Takes as inputs 
        X_test (two features)
        y_test
        fitted model
    '''
    no_targets = len(y.unique())
    
    mesh_step_size = 0.01
    x_min, x_max = X[:, 0].min() - .1, X[:, 0].max() + .1
    y_min, y_max = X[:, 1].min() - .1, X[:, 1].max() + .1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, mesh_step_size), np.arange(y_min, y_max, mesh_step_size))
    Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:,1]
    Z = Z.reshape(xx.shape)
    
    # Colors    
    #import matplotlib.colors as mcolors
    #colors = sns.color_palette("husl", no_targets)
    #cmap = mcolors.LinearSegmentedColormap.from_list("Custom", colors, 50)

    # Figure
    fig = plt.figure(figsize = (6,4))
    ax = fig.add_subplot(111)
    scats = []

    ax.scatter(X[y.values == 0,0], X[y.values == 0,1], alpha = 0.8, label = '0', color = 'blue', s = 10)
    ax.scatter(X[y.values == 1,0], X[y.values == 1,1], alpha = 0.8, label = '1', color = 'red', s = 10)

    plt.imshow(Z, interpolation = 'nearest', cmap = 'RdYlBu_r', alpha = 0.15,
               extent=(x_min, x_max, y_min, y_max), origin = 'lower')
    ax.legend()
    ax.set_aspect('equal')
    if title:
        ax.set_title(title)
    

In [7]:
def plot_confusion_matrix(y_test, y_pred, title):
    
    labelsno = len(np.unique(y_test))
    df_cm = pd.DataFrame(confusion_matrix(y_test, y_pred), index = [i for i in range(0,labelsno)],
                  columns = [i for i in range(0,labelsno)])
    
    plt.figure(figsize = (6,4))
    sns.heatmap(df_cm, annot=True)
    plt.title(title + '\nAccuracy: {0:.3f}'.format(accuracy_score(y_test, y_pred)))
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [8]:
def plot_bin_prcurve(precision, recall):
    '''
    y_test: test sample true labels
    y_clf_score: scores from decision_function
    closest_zero: clf_score closest to zero
    '''
    closest_zero = np.argmin(np.abs(thresholds))
    closest_zero_p = precision[closest_zero]
    closest_zero_r = recall[closest_zero]
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_xlim([0.0, 1.01])
    ax.set_ylim([0.0, 1.01])    
    ax.plot(precision, recall)
    ax.plot(closest_zero_p, closest_zero_r, 'o', markersize = 12, fillstyle = 'none', c = 'r', mew = 3)
    ax.set_xlabel('Precision')
    ax.set_ylabel('Recall')    
    ax.set_aspect('equal')

In [9]:
def plot_bin_ROC(y_test, y_pred):
    from sklearn.metrics import roc_curve, auc
    FPR, TPR, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(FPR, TPR)
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_xlim([-0.01, 1.00])
    ax.set_ylim([-0.01, 1.00])    
    ax.plot(FPR, TPR, lw = 3, label='ROC curve (area = {:0.3f})'.format(roc_auc))
    ax.set_xlabel('FPR')
    ax.set_ylabel('TPR')    
    ax.set_title('ROC curve')
    ax.legend(loc = 'lower right')
    ax.plot([0, 1], [0, 1], color = 'navy', lw = 3, linestyle='--')
    ax.set_aspect('equal')

In [10]:
def plot_validation_curve(train_scores, validation_scores, param_range, param_name, scoring, title = 'Validation curve'):

    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    validation_scores_mean = np.mean(validation_scores, axis=1)
    validation_scores_std = np.std(validation_scores, axis=1)    
    
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title(title)
    ax.set_xlabel(param_name)
    ax.set_ylabel("Score")
    ax.set_ylim(0.0, 1.1)
    lw = 2
    plt.semilogx(param_range, train_scores_mean, label="Training score", color="darkorange", lw=lw)
    plt.fill_between( param_range, train_scores_mean - train_scores_std
                     ,train_scores_mean + train_scores_std, alpha = 0.2
                     ,color = 'darkorange', lw = lw)
    plt.semilogx(param_range, validation_scores_mean, label="Cross-validation score", color = 'navy', lw = lw)
    plt.fill_between(param_range, validation_scores_mean - validation_scores_std,
                     validation_scores_mean + validation_scores_std, alpha = 0.2,
                     color = 'navy', lw = lw)
    ax.legend(loc = 'best')

## Visualize iris data set

In [11]:
# http://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
import matplotlib.patches as mpatches

from matplotlib.colors import ListedColormap
import math
plt.close('all')

X = iris.data[:, :2]  # for first plot only two first features 
y = iris.target
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5

# Color map
cmap = plt.cm.PuOr
colors = []
colors.append(cmap(0.3)); colors.append(cmap(0.6)); colors.append(cmap(0.9))
cmap = cmap.from_list('Custom cmap', colors, 3)

# Figure
fig = plt.figure(figsize=(9, 5))

# First axis
ax1 = fig.add_subplot(121)
ax1.scatter(X[:, 0], X[:, 1], c = y, cmap = cmap, edgecolor = 'k')
ax1.set_xlabel('Sepal length')
ax1.set_ylabel('Sepal width')
ax1.set_xlim(x_min, x_max)
ax1.set_ylim(y_min, y_max)
ax1.set_xticks(())
ax1.set_yticks(())

# legend
patch1 = mpatches.Patch(color = cmap(0), label = iris.target_names[0])
patch2 = mpatches.Patch(color = cmap(1), label = iris.target_names[1])
patch3 = mpatches.Patch(color = cmap(2), label = iris.target_names[2])
patches = [patch1, patch2, patch3]
ax1.legend(handles = patches, loc='upper right')

# To getter a better understanding of interaction of the dimensions
# plot the first three PCA dimensions
pca = PCA(n_components = 3)
pca.fit(iris.data)
X_reduced = pca.transform(iris.data)
#X_reduced = PCA(n_components=3).fit_transform(iris.data)

# Second axis
ax2 = fig.add_subplot(1, 2, 2, projection='3d')
ax2.scatter(X_reduced[:, 0], X_reduced[:, 1], X_reduced[:, 2], c=y,
           cmap=cmap, edgecolor='k', s=40)
ax2.set_title("First three PCA directions")
ax2.set_xlabel("1st eigenvector")
ax2.w_xaxis.set_ticklabels([])
ax2.set_ylabel("2nd eigenvector")
ax2.w_yaxis.set_ticklabels([])
ax2.set_zlabel("3rd eigenvector")
ax2.w_zaxis.set_ticklabels([])
ax2.view_init(azim = 110, elev = -150)

fig.tight_layout()


FigureCanvasNbAgg()

In [12]:
feature_names = list(iris.feature_names)
fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(111)
plt.imshow(pca.components_, interpolation = 'none', cmap = 'plasma')
ax.grid(None)

ax.set_xticks(np.arange(0, len(feature_names)))
ax.set_yticks(np.arange(0, 3))
ax.set_yticklabels(['First PC', 'Second PC', 'Third PC'], fontsize=12)
ax.set_title("Heatmap for features' correlations with PCs")

ax.set_xticklabels(feature_names, rotation = 20, ha='right', fontsize = 10)
plt.colorbar( ticks=[pca.components_.min(), 0, pca.components_.max()])
#ax.set_xticklabels(feature_names, rotation = 60, ha='right', fontsize = 10)
#plt.colorbar(orientation='horizontal', ticks=[pca.components_.min(), 0, pca.components_.max()], pad=0.65);

plt.subplots_adjust(left= None, bottom = 0.2, right = None, top = None, wspace = None, hspace = None);

FigureCanvasNbAgg()

## Data pre-processing

### Scikit learn bunch object into pandas data frame

In [13]:
print('Variable ''iris'' is of type ' + str(type(iris)))
columns = list(iris.feature_names ) + ['target']
iris_df = pd.DataFrame(np.concatenate((iris.data, np.array([iris.target]).T), axis=1), columns = columns)
iris_df.head()

Variable iris is of type <class 'sklearn.datasets.base.Bunch'>


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


### Splitting data into train and test sets

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Full data
y = iris_df['target']
iris_df_temp = iris_df.copy(); del iris_df_temp['target']
X = iris_df_temp
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = random_state)

# For stratified version

#X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, random_state = random_state)

### Feature scaling and normalization
See http://sebastianraschka.com/Articles/2014_about_feature_scaling.html

Heurestic: the only family of algorithms that are truly scale-invariant are tree-based methods. Even in logistic regression feature scale might play a role if the optimization algorithm used is such that it converges more quickly with normalized data

Essentially two alternatives:
 - Z-score standardization
 - minmax-scaling

Which one to use? There's no obvious answer, it depends on the application. It seems that Z-score standardizations seems to be more common approach.

Here we will have three sets: one unscaled, one z-score standardized, and one minmax-scaled.

#### Z-score standardization

In [15]:
from sklearn.preprocessing import StandardScaler

standardscaler = StandardScaler()
standardscaler.fit(X_train)

X_train_stand = standardscaler.transform(X_train)
X_test_stand = standardscaler.transform(X_test)

display_side_by_side(pd.DataFrame(X_train_stand).head(2), pd.DataFrame(X_train).head(2))

,0,1,2,3
0,0.701282,-0.850872,0.852239,0.902670
1,0.444603,-2.033225,0.390008,0.369166
,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
111,6.4,2.7,5.3,1.9
68,6.2,2.2,4.5,1.5


#### MinMax scaling

In [16]:
from sklearn.preprocessing import MinMaxScaler

minmaxscaler = MinMaxScaler()
minmaxscaler.fit(X_train)

X_train_mmscaled = minmaxscaler.transform(X_train)
X_test_mmscaled = minmaxscaler.transform(X_test)

display_side_by_side(pd.DataFrame(X_train_mmscaled).head(2), pd.DataFrame(X_train).head(2))

,0,1,2,3
0,0.583333,0.318182,0.754386,0.750000
1,0.527778,0.090909,0.614035,0.583333
,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
111,6.4,2.7,5.3,1.9
68,6.2,2.2,4.5,1.5


### Creating data sets based on dimensionality reduction
Based on the original and scaled/standardized datasets we will have following datasets:
- Full, z-score standardized dataset: X_train/X_test
- Reduced (only first two columns), minmax-scaled: X_train_mmscaled/X_test_mmscaled
- Reduced (only first two columns), z-score standardized dataset: X_train_reduced/X_test_reduced
- Reduced (first two PCs), z-score standardized dataset: X_train_pca/X_test_pca

Dependent variable y data sets will just be called y_train and y_test as they will be the same no matter how the features are given.

In [17]:
# Full data set
X_train = X_train_stand
X_test = X_test_stand

# Reduced dimension data (2 dimensions, sepal width and length), minmax scaled
X_train_mmscaled = X_train_mmscaled[:,[0,1]]
X_test_mmscaled = X_test_mmscaled[:,[0,1]]

# Reduced dimension data (2 dimensions, sepal width and length), standardized
X_train_reduced = X_train_stand[:,[0,1]]
X_test_reduced = X_test_stand[:,[0,1]]

# Reduced dimension data (2 dimensions, first two principal components)
model_pca = PCA(n_components=2).fit(X_train_stand)
X_train_pca = model_pca.transform(X_train_stand)
X_test_pca = model_pca.transform(X_test_stand)

display_side_by_side(pd.DataFrame(X_train_stand).head(2),
                     pd.DataFrame(X_train_reduced).head(2),                  
                     pd.DataFrame(X_train_pca).head(2))

0 
 1 
 2 
 3 
 
 
 
 
 0 
 0.701282 
 -0.850872 
 0.852239 
 0.902670 
 
 
 1 
 0.444603 
 -2.033225 
 0.390008 
 0.369166 
 
 
 
 
 
 
 0 
 1 
 
 
 
 
 0 
 0.701282 
 -0.850872 
 
 
 1 
 0.444603 
 -2.033225 
 
 
 
 
 
 
 0 
 1 
 
 
 
 
 0 
 1.600457 
 0.435257 
 
 
 1 
 1.219717 
 1.676353

In [18]:
# Target set where three classes have been reduced to binary case virginica vs. rest
y_train_bin = pd.DataFrame(y_train).copy()
y_test_bin = pd.DataFrame(y_test).copy()
combined = [y_train_bin, y_test_bin]
for dataset in combined:
    dataset[dataset['target'] != 2] = 0
    dataset[dataset['target'] == 2] = 1    
y_train_bin = y_train_bin['target']
y_test_bin = y_test_bin['target']

## Models

### Logistic regression

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

model_pca = LogisticRegression()
model_pca.fit(X_train_pca, y_train)
y_pred_pca = model_pca.predict(X_test_pca)

model_reduced = LogisticRegression()
model_reduced.fit(X_train_reduced, y_train)
y_pred_reduced = model_reduced.predict(X_test_reduced)

# Plot decision boundaries
plot_confusion_matrix(y_test,y_pred_pca, title = 'Conf matrix for log regression in PCA-reduced model')
title = 'Logistic regression classifier'
plot_decision_boundaries(X_test_pca, y_test, model_pca, title = title)

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [20]:
print(model.score(X_test, y_test))
print(model_pca.score(X_test_pca, y_test))

0.8947368421052632
0.7894736842105263


### K-nearest neighbours

K-nearest neighbours needs feature scaling. Since it relies on some distance measure between features, we need make sure that features are on comparable scale. Here we will use full minmac scaled feature set.

In [21]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

n_neighbors = 3

model = KNeighborsClassifier(n_neighbors = n_neighbors)
model.fit(X_train_mmscaled,y_train)
y_pred = model.predict(X_test_mmscaled)

plot_confusion_matrix(y_test,y_pred, title = 'Conf matrix for KNN model')
plot_decision_boundaries(X_test_mmscaled, y_test, model)

FigureCanvasNbAgg()

FigureCanvasNbAgg()

### Decision tree
The neat thing about decision trees is that they don't require any feature scaling. We can directly deploy non-scaled data.

Petal length and width seem to be most important features

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score, confusion_matrix
import pydotplus
import io
from scipy import misc


def show_tree(decisionTree, file_path):
    dotfile = io.StringIO()
    tree.export_graphviz(decisionTree, out_file=dotfile)
    pydotplus.graph_from_dot_data(dotfile.getvalue()).write_png(file_path)
    i = misc.imread(file_path)
    plt.imshow(i)
    
max_depth = 10 # None by default

model = DecisionTreeClassifier(max_depth = max_depth, random_state = 0)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

model_pca = DecisionTreeClassifier(max_depth = max_depth)
model_pca.fit(X_train_pca,y_train)
y_pred_pca = model_pca.predict(X_test_pca)

# Most important features in full model
res = pd.DataFrame(list(zip(X_train.columns, model.feature_importances_)))
res.columns = ['Feature', 'Importance']
res = res.sort_values(['Importance'], ascending  = False)
display(res.head())

# Write full model tree into .dot file for visualization
# .dot file can be uploaded here http://www.webgraphviz.com/
dotfile = open('dtree.dot', 'w')
tree.export_graphviz(model, out_file = dotfile, feature_names = X_train.columns)
dotfile.close()

# cannot get graphviz to work...
#show_tree(model, 'test.png')

plot_confusion_matrix(y_test,y_pred_pca, title = 'Conf matrix for log regression in PCA-reduced model')
plot_decision_boundaries(X_test_pca, y_test, model_pca, 
                         title = 'DT classifier with PCA-reduced data and max_depth = {}'.format(max_depth))

,Feature,Importance
3,petal width (cm),0.912910
2,petal length (cm),0.062510
1,sepal width (cm),0.017876
0,sepal length (cm),0.006703


FigureCanvasNbAgg()

FigureCanvasNbAgg()

### Support vector classifier

In [28]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

C = 5.0
gamma = 'auto'
kernel = 'rbf'

model = SVC(kernel = kernel, C = C, gamma = gamma, random_state = 0)
model.fit(X_train_mmscaled,y_train)
y_pred = model.predict(X_test_mmscaled)

plot_confusion_matrix(y_test,y_pred, title = 'Confusion matrix')
plot_decision_boundaries(X_test_mmscaled, y_test, model, 
                         title = 'SVC with kernel = {}, C = {}, and gamma = {}'.format(kernel,C,gamma))


FigureCanvasNbAgg()

FigureCanvasNbAgg()

### Naive Bayes Classifier

Called "naive" since they make the assumption that features are conditionally independent, given the class. That is, for all instances in a given class, the features have no correlation with each other.

Types:
 - Bernoulli: binary features
 - Multinomial: discrete features
 - Gaussian: continuous/real-valued features
 
Gaussian NB is usually used with high-dimensional data. Bernoulli and multinomial NBs are typically used for text classification where there are very large number of distinct words as features and feature vectors are sparse. NBs are easy understand and can work as baseline against more sophisticated models. Downsides include the strong assumption about conditional independency, which can lead to inferior generalization performance and inaccurate confidence estimates.

Bayes rule:
$$
\begin{align}
& \text{Posterior probability}  \ = \ \frac{\text{Prior probability}* \text{Likelihood}}{\text{Evidence}} \\[4pt]
\
\iff & Pr(y \ | \ X) = \frac{Pr(y) * Pr(X \ | \ y)}{Pr(X)}
\end{align}
$$

Bayes rule based classification model:

$$y^{*} = \underset{y}{\operatorname{argmax}} Pr(y \ | \ X) = \underset{y}{\operatorname{argmax}} Pr(y) \ Pr(X \ | \ y)$$

The "naive" assumption is that features $x_i$ are assumed to be independet of each other
$$y^{*} = \underset{y}{\operatorname{argmax}} Pr(y) \ Pr(X \ | \ y) = \underset{y}{\operatorname{argmax}} Pr(y) \ \prod_{i=1}^{n} Pr(x_i \ | \ y) $$

In [29]:
# We will use Gaussian Naive Bayes Classifier.
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix

model = GaussianNB()
model.fit(X_train_mmscaled, y_train)
y_pred = model.predict(X_test_mmscaled)

plot_confusion_matrix(y_test,y_pred, title = 'Confusion matrix')
plot_decision_boundaries(X_test_mmscaled, y_test, model, 
                         title = 'Gaussian Naive Bayes classifier')


FigureCanvasNbAgg()

FigureCanvasNbAgg()

### Random forests
Essentially, an ensemble of tress instead of one decision tree, providing more stable and better generalizable results.

Original dataset $\rightarrow$ <i>n_estimator</i> randomized bootstrapped data sets of the same size as the original data set $\rightarrow$ randomized feature split, i.e. <i>n_estimator</i> trees $\rightarrow$ ensemble prediction

Same pros as decision trees, i.e. no need for careful feature scaling. In addition, might be more robust than single decision tree, easily parallelized across multiple CPUs. Cons: difficult for humans to interpret. However, it is noteworthy that aggregated/ensemble models are not universally better than their "single" counterparts, they are better if and only if the single models suffer of instability. For example here, on the PCA-reduced non-scaled feature data single decision tree performs better.

In [30]:
from sklearn.ensemble import RandomForestClassifier

n_estimators = 10 # default = 10
max_depth = 10 # Default = None
criterion = 'gini' # default = gini, gini/entropy

model_pca = RandomForestClassifier(n_estimators = n_estimators, criterion = criterion, random_state = 0)
model_pca.fit(X_train_pca, y_train)
y_pred_pca = model_pca.predict(X_test_pca)

plot_confusion_matrix(y_test,y_pred_pca, title = 'Confusion matrix for PCA-reduced model')
plot_decision_boundaries(X_test_pca, y_test, model_pca, 
                         title = 'RF on PCA-reduced data with n_est = {}, max_depth = {} and criterion = {}'.format(n_estimators, max_depth, criterion))


C:\Users\ville_000\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


FigureCanvasNbAgg()

FigureCanvasNbAgg()

### Gradient boosted decision trees

Similar to RF in being an ensemle method, in GBDTs ensembling is done as a series of trees rather than building many trees in parallel as in RFs. Each consecutive tree is trained as to correct the mistakes of the previous tree in the series. Typically, GBDTs use "weak learners" (in this case shallow trees) build in non-stochastic way, to create a model that makes fewer and fewer mistakes down the line.

Cons of GBDT include that it requires careful tuning of learning rate and other parameters. Further, like other tree models it is not well-suited for very high dimensional sparse features 

n_estimators and learning_rate are tuned togehter: n_estimators adjusted first to exploit memory and CPUs during training, then learning_rate. max_depth usually a small vlaue (e.g. 3-5)

In [31]:
from sklearn.ensemble import GradientBoostingClassifier

n_estimators = 100 # default = 100
max_depth = 3 # default = 3
criterion = 'friedman_mse' # default = friedman_mse, mae
learning_rate = 0.1 # default = 0.1
loss = 'deviance' # default = deviance, exponential

model_pca = GradientBoostingClassifier(loss = loss, learning_rate = learning_rate, n_estimators = n_estimators, criterion = criterion, random_state = 0)
model_pca.fit(X_train_pca, y_train)
y_pred_pca = model_pca.predict(X_test_pca)

plot_confusion_matrix(y_test,y_pred_pca, title = 'Confusion matrix for PCA-reduced model')
plot_decision_boundaries(X_test_pca, y_test, model_pca, 
                         title = 'GBDT on PCA-red data: n_est = {}, md = {},\ncrit = {}, lrnrate = {}, loss = {}'
                         .format(n_estimators, max_depth, criterion, learning_rate, loss))


FigureCanvasNbAgg()

FigureCanvasNbAgg()

### Simple neural network (MLP)

Multilayer perceptron (MLP)

In [33]:
from sklearn.neural_network import MLPClassifier

solver ='lbfgs'
layers = [100, 100]
alpha = 5.0

model = MLPClassifier(hidden_layer_sizes = layers, alpha = alpha, random_state = 0, solver = solver)
model.fit(X_train_mmscaled, y_train)
y_pred = model.predict(X_test_mmscaled)

plot_confusion_matrix(y_test, y_pred, title = 'Confusion matrix for PCA-reduced model')
plot_decision_boundaries(X_test_mmscaled, y_test, model, 
                         title = 'MLP on PCA-red data: solver = {}, no_layers = {}, alpha = {}'
                         .format(solver, len(layers), alpha))

FigureCanvasNbAgg()

FigureCanvasNbAgg()

## Model evaluation

In this section we learn about how to evaluate how good of a job out classifiaction model, or classifier, does. We also learn how to optimize the choice of classifier. Below is some general information about different evaluation metrics.


Scikit learn uses following confusion matrix build in binary classification:

\begin{bmatrix}
    TN & FP \\
    FN & TP
\end{bmatrix}

<b><i>Accuracy</i></b> (Acc) is defined as

\begin{equation}
Acc = \frac{TP + TN}{TN + TP + FN + FP}
\end{equation}


<b><i>Classification error</i></b> (CErr) is defined as

\begin{equation}
CErr = 1- Acc = \frac{FP + FN}{TN + TP + FN + FP}
\end{equation}

<b><i>Recall</i></b>, also known as TPR or sentivity, exhibits the probability of detection. It ranks higher if we not only have a high number of correct positive predictions (TPs) but also avoided missing true cases (avoided FNs). Example usage: crisis detection - it is very costly to miss a crisis. It is defined as

\begin{equation}
Recall = P(\hat{Y} = 1 \ | \ Y = 1) = \frac{TP}{TP + FN}
\end{equation}

<b><i>Precision</i></b> exhibits what fraction of positive predictions are correct. This ranks higher when it is important to avoid wronly predicting a true case (avoiding FPs) and less important to have all true cases (TPs) detected. In other words, when the classifier predicts a positive class we want to be very confident that the prediction is correct. Example usage: query suggestion in a web search - it is very costy to falsely flag for a hit as customer will remember these. It is defined as

\begin{equation}
Precision = P(Y = 1 \ | \ \hat{Y} = 1) = \frac{TP}{TP + FP}
\end{equation}


<b><i>FPR</i></b>, also know as specifity, exhibits what fraction of all negative cases does the classifier incorrectly identify as positive. This metric ranks better (lower in FPR value) if we avoid falsely predicting positive cases (avoid FPs) and also manage correcly picking out negative cases (TNs). It is defined as

\begin{equation}
FPR = P(\hat{Y} = 0 \ | \ Y = 0) = \frac{FP}{TN + FP}
\end{equation}


<b>There is often a tradeoff between precision and recall</b>. We can define a measure called <b>F-score</b> that lets us determine the tradeoff between these two:

\begin{equation}
F_{\beta} = (1 + \beta^2) \  \frac{Precision \cdot Recall}{\beta^2 \cdot Precision + Recall}
\end{equation}

To weight precision and recall equally: $\beta = 1$<br>
To weight precision over recall: $\beta < 1$<br>
To weight recall over precision: $\beta > 1$<br>

Reasons why classifier accuracy is close to null accuracy baseline from DummyClassifier:

<ul style="list-style-type:circle">
  <li>Ineffective, missing, or erroneous features</li>
  <li>Hyperparameters are poorly chose</li>
  <li>Class imbalance</li>  
</ul>

One should use three data splits when running classification (or regression for that matter).
 - Training set, which is used for model building
 - Validation set, which is used for model selection/hyperparameter tuning
 - Test set, used for final evaluation
 
In practice, one achieves this by doing the following:
 - Use train_test_split once for original data, to get the test set
 - Use train_test_split (or functions that do this automatically under the hood) again on the train data from previous step to split between training and validation sets

### Select binary classifier to be used in this section
We will use a Support Vector Classifier.

In [35]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

C = 5.0
gamma = 'auto'
kernel = 'rbf'
probability = True # If True, slows down SVC

model = SVC(kernel = kernel, C = C, gamma = gamma, random_state = 0, probability = probability)
model_untrained = SVC(kernel = kernel, C = C, gamma = 'auto', random_state = 0, probability = probability)
model.fit(X_train_mmscaled,y_train_bin)
y_pred = model.predict(X_test_mmscaled)

title = 'SVC; kernel = {}, C = {}, and gamma = {}'.format(kernel,C,gamma)
plot_confusion_matrix(y_test_bin ,y_pred, title = title)
plot_decision_boundaries(X_test_mmscaled, y_test_bin, model, title = title)
plot_bin_decision_probs(X_test_mmscaled, y_test_bin, model, title = title)


FigureCanvasNbAgg()

C:\Users\ville_000\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

FigureCanvasNbAgg()

### Evaluation curves

For infor about difference between precision-recall curve and ROC curve, see https://stats.stackexchange.com/questions/7207/roc-vs-precision-and-recall-curves

#### Precision-recall curve

In [37]:
from sklearn.metrics import precision_recall_curve

y_clf_score = model.decision_function(X_test_mmscaled)
precision, recall, thresholds = precision_recall_curve(y_test_bin, y_clf_score)

plot_bin_prcurve(precision, recall)    

C:\Users\ville_000\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

#### ROC curve

In [38]:
plot_bin_ROC(y_test_bin, y_pred)     

C:\Users\ville_000\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

### Dummy classifiers

It is worthwhile to compare our classifier's performance against a dummy classifier, especially if the sample is imbalanced.  

In [39]:
from sklearn.dummy import DummyClassifier

# most_frequent, stratified, uniform
strategy = 'most_frequent'

model_dummy = DummyClassifier(strategy = strategy)
model_dummy.fit(X_train_mmscaled, y_train_bin)
y_pred_dummy = model_dummy.predict(X_test_mmscaled)

title = 'Dummy classifier using {}'.format(strategy)
plot_confusion_matrix(y_test_bin ,y_pred_dummy, title = title)
plot_decision_boundaries(X_test_mmscaled, y_test_bin, model_dummy, title = title)

C:\Users\ville_000\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

FigureCanvasNbAgg()

### Cross-validation

In [40]:
from sklearn.model_selection import cross_val_score

# accuracy, roc_auc, recall
scoring = 'accuracy'
folds = 5

print(cross_val_score(model_untrained, X_train_mmscaled, y_train_bin, cv = folds, scoring = scoring))
print(cross_val_score(model_untrained, X_train_mmscaled, y_train_bin, cv = folds, scoring = 'roc_auc'))
print(cross_val_score(model_untrained, X_train_mmscaled, y_train_bin, cv = folds, scoring = 'recall'))


[0.7826087  0.91304348 0.86956522 0.72727273 0.85714286]
[0.81666667 0.975      0.9        0.85714286 0.84183673]
[0.625      0.75       1.         0.5        0.57142857]


### Hyperparameter tuning

#### Grid search

Gris search can be used in hyperparameter tuning. It is important to realize that optimal hyperparameter value might depend on according to which metric we are optimizing over. E.g. whether we are inclined more to recall or precision.

GridSearchCV can be performed on initial training data split, as it will automatically perform a cross-validation of n folds on the input set.

In [ ]:
from sklearn.metrics.scorer import SCORERS
print(sorted(list(SCORERS.keys())))

In [41]:
# Grid search for gamma parameter in SVC
# This could be extended to multiple hyperparameters at the same time
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, recall_score, precision_score, accuracy_score

grid_values = {'gamma': [0.001, 0.01, 0.05, 0.1, 1, 10, 100]}
scoring = 'accuracy' #accuracy, precision, recall, roc_auc
ave_method = 'binary' #‘binary’ (default), ‘micro’, ‘macro’, ‘samples’, ‘weighted’

grid_model = GridSearchCV(model_untrained, param_grid = grid_values, scoring = scoring)
grid_model.fit(X_train_mmscaled, y_train_bin)
y_grid_scores = grid_model.decision_function(X_test_mmscaled)
y_grid_pred = grid_model.predict(X_test_mmscaled)

print('Grid search best parameter ({}): {}'.format(scoring,grid_model.best_params_) )
print('Grid search best score ({}): {}'.format(scoring, grid_model.best_score_))
print('-'*50)

print('Test set accuracy score: ', accuracy_score(y_test_bin, y_grid_pred))
print('Test set recall score: ', recall_score(y_test_bin, y_grid_pred, average = ave_method))
print('Test set precision score: ', precision_score(y_test_bin, y_grid_pred, average = ave_method))
print('Test set AUC score: ', roc_auc_score(y_test_bin, y_grid_scores))


Grid search best parameter (accuracy): {'gamma': 10}
Grid search best score (accuracy): 0.8125
--------------------------------------------------
Test set accuracy score:  0.7894736842105263
Test set recall score:  0.8181818181818182
Test set precision score:  0.6
Test set AUC score:  0.8181818181818182


#### Validation curve

http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.validation_curve.html
Compute scores for an estimator with different values of a specified parameter. This is similar to grid search with one parameter. However, this will also compute training scores and is merely a utility for plotting the results.

In [43]:
from sklearn.model_selection import validation_curve

param_name = 'gamma'
scoring = 'accuracy'
folds = 5
param_range = [0.001, 0.01, 0.05, 0.1, 1, 10, 100]
n_jobs = 1

train_scores, validation_scores = validation_curve( model_untrained
                                             ,X_train_mmscaled
                                             ,y_train_bin
                                             ,param_name = param_name
                                             ,param_range = param_range
                                             ,cv = folds
                                             ,scoring = scoring
                                             ,n_jobs = n_jobs
                                            )

title = 'Validation curve with method {}, folds = {}'.format(scoring, folds)
plot_validation_curve(train_scores, validation_scores, param_range, param_name, scoring, title = title)

C:\Users\ville_000\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

# Regression

## Linear regression

In [ ]:
def lin_reg_plotter(coefs, X_train, X_test, y_test, poly_order, xlims, x_points = 20):
    from sklearn.preprocessing import PolynomialFeatures
    
    x_ax0 = np.linspace(xlims[0], xlims[1], x_points).reshape(x_points, 1)
    x_ax = PolynomialFeatures(poly_order).fit_transform(x_ax0)
    y_ax = np.sum(x_ax * coefs, axis = 1).reshape(x_points, 1)

    fig, ax = plt.subplots(nrows=1, ncols=1)
    ax.scatter(X_test, y_test,  color = 'orange', label = 'test')
    ax.scatter(X_train, y_train,  color = 'purple', label = 'train')        
    ax.plot(x_ax0, y_ax, color='teal', linewidth = 3, label = 'poly = ' + str(poly_order))
    ax.legend()

In [ ]:
# inspiration from https://gist.github.com/brentp/5355925
# I would use stats rather than this...
def p_values(lin_reg_model, X, y):
        from scipy import stats
        sse = np.sum((lin_reg_model.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])        
        se = np.array([np.sqrt(np.diagonal(sse * np.linalg.inv(np.dot(X.T, X))))])
        t = lin_reg_model.coef_ / se
        p = 2 * (1 - stats.t.cdf(np.abs(t), y.shape[0] - X.shape[1]))
        return p, t

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Paramters
poly_order = 9
grid_points = 100

# Data
np.random.seed(0)
n = 15
x = np.linspace(0, 10, n) + np.random.randn(n)/5
y = np.sin(x) + x/6 + np.random.randn(n)/10

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state = 0)

# Create polynomial features into train/test data if poly_order > 1
X_train_pol = X_train.reshape(len(X_train),1)
X_test_pol = X_test.reshape(len(X_test),1)
polyfier = PolynomialFeatures(poly_order).fit(X_train_pol)
X_train_pol = polyfier.transform(X_train_pol)
X_test_pol = polyfier.transform(X_test_pol)

# Linear regression model
model_lin = LinearRegression(fit_intercept = False) # intercept included in features
model_lin.fit(X_train_pol, y_train)
y_pred_lin_test = model_lin.predict(X_test_pol)
y_pred_lin_train = model_lin.predict(X_train_pol)
print('R-squared for linear regression with poly_order = {} in train data is {:0.2f}'.format(poly_order, r2_score(y_train, y_pred_lin_train)))
print('R-squared for linear regression with poly_order = {} in test data is {:0.2f}'.format(poly_order, r2_score(y_test, y_pred_lin_test)))
print('-'*50)

# Lasso regression model
model_lasso = Lasso(alpha = 0.01, max_iter = 50000, fit_intercept = False) # intercept included in features
model_lasso.fit(X_train_pol, y_train)
y_pred_lasso_test = model_lasso.predict(X_test_pol)
y_pred_lasso_train = model_lasso.predict(X_train_pol)
print('R-squared for lasso regression with poly_order = {} in train data is {:0.2f}'.format(poly_order, r2_score(y_train, y_pred_lasso_train)))
print('R-squared for lasso regression with poly_order = {} in test data is {:0.2f}'.format(poly_order, r2_score(y_test, y_pred_lasso_test)))
print('-'*50)

# Ridge regression model
model_ridge = Ridge(alpha = 0.01, max_iter = 50000, fit_intercept = False) # intercept included in features
model_ridge.fit(X_train_pol, y_train)
y_pred_ridge_test = model_ridge.predict(X_test_pol)
y_pred_ridge_train = model_ridge.predict(X_train_pol)
print('R-squared for ridge regression with poly_order = {} in train data is {:0.2f}'.format(poly_order, r2_score(y_train, y_pred_ridge_train)))
print('R-squared for ridge regression with poly_order = {} in test data is {:0.2f}'.format(poly_order, r2_score(y_test, y_pred_ridge_test)))

#p_vals, _ = p_values(model, X_train_pol, y_train)
#print('p-values are:' )
#np.set_printoptions(precision = 3)
#print(p_vals)

lin_reg_plotter(model_lin.coef_, X_train, X_test, y_test, poly_order, [0,10], x_points = 100)
lin_reg_plotter(model_lasso.coef_, X_train, X_test, y_test, poly_order, [0,10], x_points = 100)
lin_reg_plotter(model_ridge.coef_, X_train, X_test, y_test, poly_order, [0,10], x_points = 100)

# Pipelines
Inspired by https://www.kdnuggets.com/2017/12/managing-machine-learning-workflows-scikit-learn-pipelines-part-1.html

In [22]:
# train adn test data
y = iris_df['target']
iris_df_temp = iris_df.copy(); del iris_df_temp['target']
X = iris_df_temp
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = random_state)

In [ ]:
# Psossible debuggin class to extract transformed training data
# https://stackoverflow.com/questions/48743032/get-intermediate-data-state-in-scikit-learn-pipeline

# Pipeline does not support transformations to your target 
# https://stackoverflow.com/questions/31259891/put-customized-functions-in-sklearn-pipeline

from sklearn.base import BaseEstimator, TransformerMixin
class Debug(BaseEstimator, TransformerMixin):

    def transform(self, X):
        # Return X data
        self.X = X
        return X

    def fit(self, X, y=None, **fit_params):
        return self


In [23]:
# Pipeline with stadard scaling of data, extracting first two PCAs, and logistic regression classifier
from sklearn.pipeline import Pipeline
pipe_lr = Pipeline([
                    ('scl', StandardScaler()),
                    ('pca', PCA(n_components=2, random_state = random_state)),
                    #("debug", Debug()),
                    ('clf', LogisticRegression(random_state=random_state))])


In [24]:
# Train pipeline
pipe_lr.fit(X_train, y_train)

# Predict using trained pipeline
# Predict imposes same tranformations to the test set as have been used for train data in the pipeline
# https://youtu.be/URdnFlZnlaE?t=479
y_pred_pca = pipe_lr.predict(X_test)

I don't know how to extract transformed X_test from pipe_lr.predict(X_test), so we cannot draw the decision regions like in the example without a pipeline. From confusion matrix and accuracy we see, however, that the results coincide.

In [25]:
plot_confusion_matrix(y_test,y_pred_pca, title = 'Conf matrix for log regression in PCA-reduced model')

#title = 'Logistic regression classifier'
#plot_decision_boundaries(X_test.values, y_test, model_lr, title = title)

FigureCanvasNbAgg()

# Stuff to add


Pipelines<br>
Linear SVMs<br>
Kernelized SVMs<br>


Treating imbalanced datasets in classification<br>

Different classification scoring metrics in action: classification_report. This especially for multi-class case, not just binary<br>
Micro and macro averages in multi-class classification<br>

Unsupervised learning: k-means clustering, manifold learning, agglomerative clustering, t-sne, dbscan 
